<a href="https://colab.research.google.com/github/aubreymoore/crb-damage-detector-colab/blob/main/detect_and_annotate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab Jupyter is still in development and may change a lot or even fail.

If you have any problems, questions, or suggestions please contact me.

Aubrey Moore (aubreymoore2013@gmail.com)

In [160]:
# set up a timer for each cell
%pip install ipython-autotime -q
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.01 s (started: 2025-10-22 09:37:10 +00:00)


# detect_and_annotate.ipynb

NOTE: The following documentation is already slightly out of date.
Please visit https://github.com/aubreymoore/crb-damage-detector-colab before running this notebook for the first time.

This Colab Jupyter notebook runs a custom YOLOv8 object detector which scans images to find three object classes: live coconut palms, dead coconut palms and v-shaped cuts symptomatic of damage caused by coconut rhinoceros beetle, *Oryctes rhinoceros*.

IMPORTANT: Shortly after the MAIN PROGRAM section begins executing, a BROWSE button will appear below the active cell to allow you to upload single file of input data from your loacal machine to Colab.

**Note that Colab will just sit there and not do anything until you have entered a path to a test file of URLs or a ZIP file of images on your local machine.** [Click here to scroll down to the "Browser" button.](#scrollTo=5zSjfTXvIv2q&line=1&uniqifier=1)

You may choose between 2 options:
* A TEXT file (\*.txt) containing URLs for images to be scanned. One URL per line. (This is the most efficient option.)
* A ZIP file (\*.zip) containing images to be scanned.


Test data are available in a companion GitHub repository at https://github.com/aubreymoore/crb-damage-detector-colab. To use the test data, download it to your local computer it as a [ZIP file](https://github.com/aubreymoore/crb-damage-detector-colab/archive/refs/heads/main.zip) and unzip it. If you have **git** installed, you can clone the repo as an alternative. The TEXT file or ZIP file to be uploaded to Colab will be found in the repository's **data** folder.

To scan images, select **Runtime | Run all** on the main menu.
Results will be in a temporary OUTPUT folder which you can access using the **File browser** in the left Colab panel.

When image scanning is complete, the OUTPUT folder will be compressed into a single ZIP file and automatically downloaded to your computer.

### TODO

- [ finished 2024-10-19] Reduce size of images in the companion GH repo to max dimension of 960px
- [ ] Copy current trained model to companion GH repo
- [ ] Copy this Jupyter notebook to companion GH repo
- [ ] Add confidence values to bounding box labels.
- [ ] Add database to OUTPUT folder
- [ ] Extract GPS coordinates from image files
- [ ] Figure out how to use URLs to access images stored on OneDrive (Sharepoint)

The following cell contains a link to text file which contains a list of image files to be scanned.

For example,
```https://github.com/aubreymoore/crb-damage-detector-colab/raw/refs/heads/main/data/urls.txt``` links to a file which contains:
```
https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0532.JPG?raw=true
https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0671.JPG?raw=true
https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_06XX.JPG?raw=true
https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0695.JPG?raw=true
https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0704.JPG?raw=true
https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0713.JPG?raw=true
```
Edit the following so that the url points to your own data, then press ```Run All``` near the top of this page, and ignore the warning.

# How to Import Images to be Scanned for CRB Damage

This notebook currently uses a single parameter named INPUT_DATA_URL which is  a hyperlink  that determines which images are imported to be scanned for CRB damage. INPUT_DATA_URL points to **a text file containing a list of hyperlinks to images** or **a zip file containing the actual images**.

#### Text file example
Here is a link to a text file which points to a list of 5 images of coconut palm images from from Vanuatu provided by Sulav Paudel at AgResearch New Zealand.

```
INPUT_DATA_URL =  'https://github.com/aubreymoore/crb-damage-detector-colab/raw/refs/heads/main/Vanuatu.txt'
````

[View text file contents.](https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/Vanuatu.txt)

#### Zip file example
Here is an example of a zip file containing 3 images of coconut palms from Maui provided by  Nicole Ferguson at Hawaii Department of Agriculture.

```
INPUT_DATA_URL = 'https://github.com/aubreymoore/crb-damage-detector-colab/raw/refs/heads/main/vcuts_maui.zip'
````

[Download the zip file to your computer.](https://github.com/aubreymoore/crb-damage-detector-colab/raw/refs/heads/main/vcuts_maui.zip)


In [161]:
# The last uncommented line will determine where the source of images to be scanned.

#  uncomment the following line to scan images in listed in Vanuatu.txt
# INPUT_DATA_URL = 'https://github.com/aubreymoore/crb-damage-detector-colab/raw/refs/heads/main/Vanuatu.txt'
# INPUT_DATA_URL = 'https://aubreymoore.github.io/CRB-FIDL/image-list.txt'

# uncomment the following line to scan images in vcuts_maui.zip
# INPUT_DATA_URL = 'https://github.com/aubreymoore/crb-damage-detector-colab/raw/refs/heads/main/vcuts_maui.zip'
INPUT_DATA_URL = 'https://github.com/aubreymoore/crb-damage-detection-models/raw/refs/heads/main/datasets/CRB003s/images/val/first10_fromCRB003s_val.zip'

time: 636 µs (started: 2025-10-22 09:37:15 +00:00)


# Load Python packages which are not preinstalled by Colab

In [162]:
%pip install ultralytics -q
%pip install supervision -q
%pip install imutils -q
%pip install icecream -q

time: 36.6 s (started: 2025-10-22 09:37:15 +00:00)


# Import modules

In [163]:
import cv2
import supervision as sv
from ultralytics import YOLO
import imutils
import glob
import os
import shutil
from skimage import io
from icecream import ic
from google.colab import files
# ultralytics.checks()

time: 757 µs (started: 2025-10-22 09:37:52 +00:00)


# Define functions

In [164]:
def get_list_from_url(url):
  """
  Downloads a text file from a URL and returns a list of its lines.
  """
  try:
    # Download the file
    !wget -q -O temp.txt {url}
    # Read the lines into a list
    with open('temp.txt', 'r') as f:
      lines = f.read().splitlines()
    # Clean up the temporary file
    os.remove('temp.txt')
    return lines
  except Exception as e:
    print(f"Error downloading or reading file from {url}: {e}")
    return None

# # Example usage (replace with actual URL)
# url = 'https://github.com/aubreymoore/crb-damage-detector-colab/raw/refs/heads/main/data/urls.txt'
# my_list = get_list_from_url(url)
# if my_list:
#   print("List created from URL:")
#   print(my_list)
# else:
#   print("Failed to create list from URL.")

time: 935 µs (started: 2025-10-22 09:37:52 +00:00)


In [165]:
def upload_model_weights():
  '''
  Upload model weights from GitHub repo to **weights.pt** only if this file does not already exist.
  '''
  # !wget -nc https://github.com/aubreymoore/code-for-CRB-damage-ai/raw/refs/heads/main/models/3class/train5/weights/best.pt -O weights.pt
  !wget -nc https://github.com/aubreymoore/crb-damage-detection-models/raw/refs/heads/main/models/CRB003/weights/CRB003.pt -O weights.pt

# upload_model_weights()

time: 857 µs (started: 2025-10-22 09:37:52 +00:00)


In [166]:
def load_model_weights():
  model = YOLO('weights.pt')

time: 735 µs (started: 2025-10-22 09:37:52 +00:00)


In [167]:
def create_input_folder():
  if not os.path.exists('INPUT'):
    os.makedirs('INPUT')

# create_input_folder()

time: 1.43 ms (started: 2025-10-22 09:37:52 +00:00)


In [168]:
def create_output_folder():
  if not os.path.exists('OUTPUT'):
    os.makedirs('OUTPUT')

# create_output_folder()

time: 533 µs (started: 2025-10-22 09:37:52 +00:00)


In [169]:
def run_garbage_disposal():
  '''
  Delete any data files left over from the last run.
  '''
  shutil.rmtree('INPUT', ignore_errors=True)
  shutil.rmtree('OUTPUT', ignore_errors=True)
  shutil.rmtree('sample_data', ignore_errors=True)

  try:
    os.remove('weights.pt')
  except OSError:
    pass

# run_garbage_disposal()

time: 817 µs (started: 2025-10-22 09:37:52 +00:00)


In [170]:
def upload_and_unpack_zip_or_txt():
  '''
  Upload images (*.zip) or list of URLs (*.txt)
  '''
  uploaded = files.upload(target_dir='INPUT')
  filename = list(uploaded.keys())[0]

  urls = None
  image_file_dir = None

  if filename.endswith('.txt'):
    input_mode = 'text'
    with open(filename, 'r') as f:
      urls = f.read().splitlines()
  elif filename.endswith('.zip'):
    input_mode = 'zip'
    !unzip -q $filename -d INPUT
    image_file_dir = f'INPUT/{filename}'.replace('.zip', '')
    ic(image_file_dir)
  else:
    raise ValueError('INPUT file must be *.txt or *.zip.')
  return input_mode, urls, image_file_dir

# input_mode, urls, image_file_dir = upload_and_unpack_zip_or_txt()
# ic(input_mode)
# ic(urls)
# ic(image_file_dir)

time: 1.09 ms (started: 2025-10-22 09:37:52 +00:00)


In [171]:
def get_input_file_list():
  return glob.glob(f'INPUT/**/*', recursive=True)

# get_input_file_list()

time: 472 µs (started: 2025-10-22 09:37:52 +00:00)


In [172]:
def detect_objects(image, model, box_annotator, label_annotator, csv_sink, mask_annotator):
  '''
  detect objects in an image
  returns detections and an annotated image
  '''
  results = model(image)[0]
  detections = sv.Detections.from_ultralytics(results)
  # ic(detections)
  annotated_image = box_annotator.annotate(image, detections=detections)
  annotated_image = mask_annotator.annotate(scene=annotated_image, detections=detections)
  labels = [f"{model.model.names[class_id]} {confidence:.2f}" for class_id, confidence in zip(detections.class_id, detections.confidence)]
  annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
  return detections, annotated_image

time: 1.01 ms (started: 2025-10-22 09:37:52 +00:00)


In [173]:
import requests
import zipfile
import os

def download_and_unzip(url, output_dir):
  """
  Downloads a zip file from a URL and unzips it to a specified directory.

  Args:
    url: The URL of the zip file.
    output_dir: The directory to extract the contents to.
  """

  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  local_zip_path = os.path.join(output_dir, url.split('/')[-1])

  try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(local_zip_path, 'wb') as f:
      for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
      zip_ref.extractall(output_dir)

    os.remove(local_zip_path) # Clean up the downloaded zip file

  except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")
  except zipfile.BadZipFile:
    print("Error: The downloaded file is not a valid zip file.")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Example usage (uncomment to test)
# zip_url = 'https://github.com/aubreymoore/crb-damage-detector-colab/raw/refs/heads/main/vcuts_maui.zip'
# output_directory = 'downloaded_zip_contents'
# download_and_unzip(zip_url, output_directory)

time: 2.46 ms (started: 2025-10-22 09:37:52 +00:00)


# MAIN PROGRAM

In [174]:
# Clear data files from previous run
run_garbage_disposal()

create_input_folder()
create_output_folder()

# Upload images or list of URLs
# input_mode, urls, image_file_dir = upload_and_unpack_zip_or_txt()


# Upload weights from trained model and load them
upload_model_weights()
model = YOLO('weights.pt')

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
mask_annotator = sv.MaskAnnotator()    #NEW
csv_sink = sv.CSVSink('OUTPUT/detections.csv')
csv_sink.open()

# Scan images
if INPUT_DATA_URL[-4:].lower() == '.txt':
  urls = get_list_from_url(INPUT_DATA_URL)
  ic(urls)
  for url in urls:
    try:
      image = imutils.url_to_image(url)
      detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink, mask_annotator)
      csv_sink.append(
          detections,
          custom_data={'image_h': image.shape[0], 'image_w': image.shape[1], 'source': url}
      )

      # Extract filename from URL
      filename = url.split('/')[-1]
      pos = filename.find('?')
      if pos >= 0:
        filename = filename[:pos]
      output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
      ic(output_path)
      os.makedirs(os.path.dirname(output_path), exist_ok = True)
      cv2.imwrite(output_path, annotated_image)
    except:
      print(f'Error processing {url}')
    continue

if INPUT_DATA_URL[-4:].lower() == '.zip':
  download_and_unzip(url=INPUT_DATA_URL, output_dir='INPUT')
  input_file_list = get_input_file_list()
  ic(input_file_list)
  for image_path in input_file_list:
    ic(image_path)
    try:
      image = cv2.imread(image_path)


      detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink, mask_annotator)
      csv_sink.append(
          detections,
          custom_data={'image_h': image.shape[0], 'image_w': image.shape[1], 'source': image_path}
      )

      filename = os.path.basename(image_path)
      output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
      os.makedirs(os.path.dirname(output_path), exist_ok = True)
      result = cv2.imwrite(output_path, annotated_image)
    except:
      print(f'Error processing {image_path}')
    continue

csv_sink.close()

--2025-10-22 09:37:52--  https://github.com/aubreymoore/crb-damage-detection-models/raw/refs/heads/main/models/CRB003/weights/CRB003.pt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aubreymoore/crb-damage-detection-models/refs/heads/main/models/CRB003/weights/CRB003.pt [following]
--2025-10-22 09:37:52--  https://raw.githubusercontent.com/aubreymoore/crb-damage-detection-models/refs/heads/main/models/CRB003/weights/CRB003.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20598820 (20M) [application/octet-stream]
Saving to: ‘weights.pt’

weights.pt          100%[===================>]  19.64M  

ic| input_file_list: ['INPUT/20210903_104016.jpg',
                      'INPUT/20210903_110601.jpg',
                      'INPUT/20210903_110556.jpg',
                      'INPUT/20210903_115117.jpg',
                      'INPUT/20220218_103856.jpg',
                      'INPUT/20210903_125537.jpg',
                      'INPUT/20210830_102852.jpg',
                      'INPUT/20210902_125437.jpg',
                      'INPUT/20210824_094010.jpg',
                      'INPUT/20210903_110550.jpg']
ic| image_path: 'INPUT/20210903_104016.jpg'


0: 544x960 1 dead, 1929.2ms
Speed: 6.6ms preprocess, 1929.2ms inference, 17.8ms postprocess per image at shape (1, 3, 544, 960)



ic| image_path: 'INPUT/20210903_110601.jpg'


0: 544x960 3 deads, 2730.4ms
Speed: 5.4ms preprocess, 2730.4ms inference, 41.5ms postprocess per image at shape (1, 3, 544, 960)



...

0: 544x960 1 damaged, 3 deads, 1 vcut, 928.2ms
Speed: 4.8ms preprocess, 928.2ms inference, 18.7ms postprocess per image at shape (1, 3, 544, 960)



ic| image_path: 'INPUT/20210824_094010.jpg'


0: 544x960 1 damaged, 1 dead, 1 vcut, 1029.5ms
Speed: 4.9ms preprocess, 1029.5ms inference, 24.6ms postprocess per image at shape (1, 3, 544, 960)



ic| image_path: 'INPUT/20210903_110550.jpg'


0: 544x960 6 deads, 1425.8ms
Speed: 5.5ms preprocess, 1425.8ms inference, 39.3ms postprocess per image at shape (1, 3, 544, 960)
time: 21 s (started: 2025-10-22 09:37:52 +00:00)


In [175]:
# Clear data files from previous run
run_garbage_disposal()

create_input_folder()
create_output_folder()

# Upload images or list of URLs
# input_mode, urls, image_file_dir = upload_and_unpack_zip_or_txt()


# Upload weights from trained model and load them
upload_model_weights()
model = YOLO('weights.pt')

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
mask_annotator = sv.MaskAnnotator()    #NEW
csv_sink = sv.CSVSink('OUTPUT/detections.csv')
csv_sink.open()

# Scan images
if INPUT_DATA_URL[-4:].lower() == '.txt':
  urls = get_list_from_url(INPUT_DATA_URL)
  ic(urls)
  for url in urls:
    try:
      image = imutils.url_to_image(url)
      detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink, mask_annotator)
      csv_sink.append(
          detections,
          custom_data={'image_h': image.shape[0], 'image_w': image.shape[1], 'source': url}
      )

      # Extract filename from URL
      filename = url.split('/')[-1]
      pos = filename.find('?')
      if pos >= 0:
        filename = filename[:pos]
      output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
      ic(output_path)
      os.makedirs(os.path.dirname(output_path), exist_ok = True)
      cv2.imwrite(output_path, annotated_image)
    except:
      print(f'Error processing {url}')
    continue

if INPUT_DATA_URL[-4:].lower() == '.zip':
  download_and_unzip(url=INPUT_DATA_URL, output_dir='INPUT')
  input_file_list = get_input_file_list()
  ic(input_file_list)
  for image_path in input_file_list:
    ic(image_path)
    try:
      image = cv2.imread(image_path)
      detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink, mask_annotator)
      csv_sink.append(
          detections,
          custom_data={'image_h': image.shape[0], 'image_w': image.shape[1], 'source': image_path}
      )

      filename = os.path.basename(image_path)
      output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
      os.makedirs(os.path.dirname(output_path), exist_ok = True)
      result = cv2.imwrite(output_path, annotated_image)
    except:
      print(f'Error processing {image_path}')
    continue

csv_sink.close()

--2025-10-22 09:38:13--  https://github.com/aubreymoore/crb-damage-detection-models/raw/refs/heads/main/models/CRB003/weights/CRB003.pt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aubreymoore/crb-damage-detection-models/refs/heads/main/models/CRB003/weights/CRB003.pt [following]
--2025-10-22 09:38:13--  https://raw.githubusercontent.com/aubreymoore/crb-damage-detection-models/refs/heads/main/models/CRB003/weights/CRB003.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20598820 (20M) [application/octet-stream]
Saving to: ‘weights.pt’

weights.pt          100%[===================>]  19.64M  

ic| input_file_list: ['INPUT/20210903_104016.jpg',
                      'INPUT/20210903_110601.jpg',
                      'INPUT/20210903_110556.jpg',
                      'INPUT/20210903_115117.jpg',
                      'INPUT/20220218_103856.jpg',
                      'INPUT/20210903_125537.jpg',
                      'INPUT/20210830_102852.jpg',
                      'INPUT/20210902_125437.jpg',
                      'INPUT/20210824_094010.jpg',
                      'INPUT/20210903_110550.jpg']
ic| image_path: 'INPUT/20210903_104016.jpg'



0: 544x960 1 dead, 1478.3ms
Speed: 7.4ms preprocess, 1478.3ms inference, 8.6ms postprocess per image at shape (1, 3, 544, 960)



ic| image_path: 'INPUT/20210903_110601.jpg'


0: 544x960 3 deads, 2356.1ms
Speed: 5.7ms preprocess, 2356.1ms inference, 25.0ms postprocess per image at shape (1, 3, 544, 960)



...

0: 544x960 1 damaged, 3 deads, 1 vcut, 968.1ms
Speed: 5.5ms preprocess, 968.1ms inference, 19.5ms postprocess per image at shape (1, 3, 544, 960)



ic| image_path: 'INPUT/20210824_094010.jpg'


0: 544x960 1 damaged, 1 dead, 1 vcut, 1003.5ms
Speed: 5.0ms preprocess, 1003.5ms inference, 18.5ms postprocess per image at shape (1, 3, 544, 960)



ic| image_path: 'INPUT/20210903_110550.jpg'


0: 544x960 6 deads, 967.7ms
Speed: 5.3ms preprocess, 967.7ms inference, 37.9ms postprocess per image at shape (1, 3, 544, 960)
time: 19.8 s (started: 2025-10-22 09:38:13 +00:00)


## Please click on the Browse buttom when it appears above this cell.

### Download OUTPUT folder as a ZIP file

In [176]:
!zip -r OUTPUT.zip OUTPUT

updating: OUTPUT/ (stored 0%)
updating: OUTPUT/detections.csv (deflated 64%)
updating: OUTPUT/20210830_102852_annotated.jpg (deflated 2%)
updating: OUTPUT/20210903_115117_annotated.jpg (deflated 2%)
updating: OUTPUT/20220218_103856_annotated.jpg (deflated 2%)
updating: OUTPUT/20210902_125437_annotated.jpg (deflated 3%)
updating: OUTPUT/20210903_104016_annotated.jpg (deflated 2%)
updating: OUTPUT/20210903_110550_annotated.jpg (deflated 2%)
updating: OUTPUT/20210903_125537_annotated.jpg (deflated 2%)
updating: OUTPUT/20210903_110601_annotated.jpg (deflated 2%)
updating: OUTPUT/20210824_094010_annotated.jpg (deflated 2%)
updating: OUTPUT/20210903_110556_annotated.jpg (deflated 2%)
time: 512 ms (started: 2025-10-22 09:38:33 +00:00)


In [177]:
from google.colab import files
files.download("OUTPUT.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 7.24 ms (started: 2025-10-22 09:38:34 +00:00)


# FINISHED
If everything worked as intended, you should find a file named **OUTPUT.zip** in your Downloads folder. Unzip this file to see results.

In [178]:

print('FINISHED')

FINISHED
time: 614 µs (started: 2025-10-22 09:38:34 +00:00)
